In [ ]:
from dotenv import load_dotenv

import clears
import sheets

load_dotenv()

In [ ]:
gc = sheets.authenticate_gspread()

In [ ]:
from sheets import load_cld_from_main_sheet

t = load_cld_from_main_sheet(gc)

In [ ]:
# first map row = 10 (index 9)
# 8 star = cols B to F (1:6)
# 7 star = cols H to L (7:12)
# 6 star = cols N to R (13:18)
# (8-n) star = cols ? to ? (6n+i : 6n+6)
star_groups = [
    [r[6*i + 1 : 6*i + 4] for r in t[9:] if r[6*i + 1]]
    for i in range(8)
]

In [ ]:
golden_rows = [row for group in star_groups for row in group]

In [ ]:
def trim_map_name(map_name):
    map_name_no_author = map_name.replace("\n", " ").replace("  ", " ").split(" by ")[0]
    clear_type = "[C]"  # default
    trimmed_map_name = map_name_no_author
    if map_name_no_author.endswith("]"):
        i = map_name_no_author.rfind("[")
        clear_type = map_name_no_author[i:]
        trimmed_map_name = map_name_no_author[:i - 1]
    return trimmed_map_name, clear_type

In [ ]:
def str_to_tier(s):
    tier = 0 # default for untiered/undetermined
    if s.startswith("Tier"):
        space_index = s.rindex(" ")
        tier_str = s[space_index + 1:]
        tier = int(tier_str)
    return tier

In [ ]:
def populate_golden_tier(golden_tiers, trimmed_map_name, index, new_tier_str):
    tier_list = golden_tiers[trimmed_map_name]
    if new_tier_str.startswith("Tier") or new_tier_str in ["Undetermined", "Untiered"]:
        if tier_list[index] is None:
            tier_list[index] = new_tier_str
        elif tier_list[index] != new_tier_str:
            old_tier = str_to_tier(tier_list[index])
            new_tier = str_to_tier(new_tier_str)
            if old_tier < new_tier:
                print(f"WARNING: {trimmed_map_name} [{"C" if index == 0 else "FC"}] was {tier_list[index]}, overriding with {new_tier_str}")
                tier_list[index] = new_tier_str
            elif old_tier > new_tier:
                print(f"WARNING: {trimmed_map_name} [{"C" if index == 0 else "FC"}] already was {tier_list[index]}, ignoring {new_tier_str}")

In [ ]:
golden_tiers = {}

for row in golden_rows:
    map_name, c_tier, fc_tier = row
    trimmed_map_name, clear_type = trim_map_name(map_name)
    if clear_type == "[FC]":
        c_tier, fc_tier = "", c_tier
    elif not fc_tier or fc_tier == "<<<":
        fc_tier = c_tier

    if trimmed_map_name not in golden_tiers:
        golden_tiers[trimmed_map_name] = [None, None]

    populate_golden_tier(golden_tiers, trimmed_map_name, 0, c_tier)
    populate_golden_tier(golden_tiers, trimmed_map_name, 1, fc_tier)

WARNING: Crystal Garden [C] was Tier 14, overriding with Tier 16

WARNING: Crystal Garden [FC] was Tier 14, overriding with Tier 16

WARNING: Magnetic Dawn (B-Side) [C] already was Tier 7, ignoring Undetermined

WARNING: Magnetic Dawn (B-Side) [FC] already was Tier 7, ignoring Undetermined

WARNING: Mauve [C] was Tier 4, overriding with Tier 8

WARNING: Mauve [FC] was Tier 4, overriding with Tier 10

WARNING: Mauve (Old) [C] was Tier 6, overriding with Tier 8

WARNING: Mauve (Old) [FC] was Tier 6, overriding with Tier 11

In [ ]:
current_clears_sheet = sheets.load_current_clears_from_main_sheet(gc)
current_state, map_difficulties = clears.get_current_state_and_maps_from_sheet_values(current_clears_sheet)

In [ ]:
for map_name in map_difficulties:
    trimmed_map_name, clear_type = trim_map_name(map_name)
    if trimmed_map_name not in golden_tiers:
        print(f"WARNING: {trimmed_map_name} doesn't have a golden tier")

so all goldens seem to be consistent and calculable given a map name, apart from cg, mbwn, mauve, and omauve. cg has multiple versions which are the same hist difficulty but different molden tiers. mbwn has normal and no heart death versions which are (again) the same hist difficulty but different molden tiers. mauve and omauve each have skipful and skipless routes which are (again) the same hist difficulty but different molden tiers. with the way we track goldens, there is no way to differentiate between these without querying goldberries for the corresponding golden submission, which i'm considering out of scope. instead, we will write something like "either tier 6 or tier 8" in the notification if we're unsure, or just assume the higher tier to be nice.